In [1]:
import tensorflow as tf
import numpy as np
from tensorflow_io.bigquery import BigQueryClient
import datetime
print(tf.__version__)

1.15.2


In [2]:
import tensorflow_io

In [3]:
tensorflow_io.__version__

'0.8.1'

## Read data

In [4]:
PROJECT_ID = "event-driven-ml"
DATASET_GCP_PROJECT_ID = "event-driven-ml"
DATASET_ID = "edml_nyc_yellow_taxi_us"
TABLE_ID = "gis_feat_eng"

In [5]:
INPUT_COLUMNS = ["uuid", "dayofweek", "hourofday", "weekofyear", "pickup_zone_name", "dropoff_zone_name", "passenger_count", "distance", "trip_duration"]
LABEL_COLUMN = "trip_duration"
KEY_COLUMN = "uuid"

# Set default values for each CSV column
DEFAULTS = [['no_key'], [1], [0], [54], [""], [""], [1], [0], []]

NEMBEDS = 10

In [6]:
VOCABULARY = [
    'Allerton/Pelham Gardens', 'Alphabet City', 'Arden Heights', 'Arrochar/Fort Wadsworth', 
    'Astoria', 'Astoria Park', 'Auburndale', 'Baisley Park', 'Bath Beach', 'Battery Park',
    'Battery Park City', 'Bay Ridge', 'Bay Terrace/Fort Totten', 'Bayside', 'Bedford', 
    'Bedford Park', 'Bellerose', 'Belmont', 'Bensonhurst East', 'Bensonhurst West',
    'Bloomfield/Emerson Hill', 'Bloomingdale', 'Boerum Hill', 'Borough Park', 
    'Breezy Point/Fort Tilden/Riis Beach', 'Briarwood/Jamaica Hills', 'Brighton Beach',
    'Broad Channel', 'Bronx Park', 'Bronxdale', 'Brooklyn Heights', 'Brooklyn Navy Yard', 
    'Brownsville', 'Bushwick North', 'Bushwick South', 'Cambria Heights', 'Canarsie', 'Carroll Gardens',
    'Central Harlem', 'Central Harlem North', 'Central Park', 'Charleston/Tottenville', 'Chinatown',
    'City Island', 'Claremont/Bathgate', 'Clinton East', 'Clinton Hill', 'Clinton West', 'Co-Op City',
    'Cobble Hill', 'College Point', 'Columbia Street', 'Coney Island', 'Corona', 'Country Club', 'Crotona Park',
    'Crotona Park East', 'Crown Heights North', 'Crown Heights South', 'Cypress Hills', 'DUMBO/Vinegar Hill',
    'Douglaston', 'Downtown Brooklyn/MetroTech', 'Dyker Heights', 'East Chelsea', 'East Concourse/Concourse Village',
    'East Elmhurst', 'East Flatbush/Farragut', 'East Flatbush/Remsen Village', 'East Flushing', 'East Harlem North',
    'East Harlem South', 'East New York', 'East New York/Pennsylvania Avenue', 'East Tremont', 'East Village',
    'East Williamsburg', 'Eastchester', 'Elmhurst', 'Elmhurst/Maspeth', "Eltingville/Annadale/Prince's Bay", 'Erasmus',
    'Far Rockaway', 'Financial District North', 'Financial District South', 'Flatbush/Ditmas Park', 'Flatiron',
    'Flatlands', 'Flushing', 'Flushing Meadows-Corona Park', 'Fordham South', 'Forest Hills',
    'Forest Park/Highland Park', 'Fort Greene', 'Fresh Meadows', 'Freshkills Park', 'Garment District', 'Glen Oaks',
    'Glendale', 'Gowanus', 'Gramercy', 'Gravesend', 'Great Kills', 'Great Kills Park', 'Green-Wood Cemetery',
    'Greenpoint', 'Greenwich Village North', 'Greenwich Village South', 'Grymes Hill/Clifton', 'Hamilton Heights',
    'Hammels/Arverne', 'Heartland Village/Todt Hill', 'Highbridge', 'Highbridge Park', 'Hillcrest/Pomonok', 'Hollis',
    'Homecrest', 'Howard Beach', 'Hudson Sq', 'Hunts Point', 'Inwood', 'Inwood Hill Park', 'JFK Airport',
    'Jackson Heights', 'Jamaica', 'Jamaica Bay', 'Jamaica Estates', 'Kensington', 'Kew Gardens', 'Kew Gardens Hills',
    'Kingsbridge Heights', 'Kips Bay', 'LaGuardia Airport', 'Laurelton', 'Lenox Hill East', 'Lenox Hill West',
    'Lincoln Square East', 'Lincoln Square West', 'Little Italy/NoLiTa', 'Long Island City/Hunters Point',
    'Long Island City/Queens Plaza', 'Longwood', 'Lower East Side', 'Madison', 'Manhattan Beach', 'Manhattan Valley',
    'Manhattanville', 'Marble Hill', 'Marine Park/Floyd Bennett Field', 'Marine Park/Mill Basin', 'Mariners Harbor',
    'Maspeth', 'Meatpacking/West Village West', 'Melrose South', 'Middle Village', 'Midtown Center', 'Midtown East',
    'Midtown North', 'Midtown South', 'Midwood', 'Morningside Heights', 'Morrisania/Melrose', 'Mott Haven/Port Morris',
    'Mount Hope', 'Murray Hill', 'Murray Hill-Queens', 'New Dorp/Midland Beach', 'Newark Airport', 'North Corona',
    'Norwood', 'Oakland Gardens', 'Oakwood', 'Ocean Hill', 'Ocean Parkway South', 'Old Astoria', 'Ozone Park',
    'Park Slope', 'Parkchester', 'Pelham Bay', 'Pelham Bay Park', 'Pelham Parkway', 'Penn Station/Madison Sq West',
    'Port Richmond', 'Prospect Heights', 'Prospect Park', 'Prospect-Lefferts Gardens', 'Queens Village',
    'Queensboro Hill', 'Queensbridge/Ravenswood', 'Randalls Island', 'Red Hook', 'Rego Park', 'Richmond Hill',
    'Ridgewood', 'Rikers Island', 'Riverdale/North Riverdale/Fieldston', 'Rockaway Park', 'Roosevelt Island', 'Rosedale',
    'Rossville/Woodrow', 'Saint Albans', 'Saint George/New Brighton', 'Saint Michaels Cemetery/Woodside',
    'Schuylerville/Edgewater Park', 'Seaport', 'Sheepshead Bay', 'SoHo', 'Soundview/Bruckner', 'Soundview/Castle Hill',
    'South Beach/Dongan Hills', 'South Jamaica', 'South Ozone Park', 'South Williamsburg', 'Springfield Gardens North',
    'Springfield Gardens South', 'Spuyten Duyvil/Kingsbridge', 'Stapleton', 'Starrett City', 'Steinway',
    'Stuy Town/Peter Cooper Village', 'Stuyvesant Heights', 'Sunnyside', 'Sunset Park East', 'Sunset Park West',
    'Sutton Place/Turtle Bay North', 'Times Sq/Theatre District', 'TriBeCa/Civic Center', 'Two Bridges/Seward Park',
    'UN/Turtle Bay South', 'Union Sq', 'University Heights/Morris Heights', 'Upper East Side North',
    'Upper East Side South', 'Upper West Side North', 'Upper West Side South', 'Van Cortlandt Park',
    'Van Cortlandt Village', 'Van Nest/Morris Park', 'Washington Heights North', 'Washington Heights South',
    'West Brighton', 'West Chelsea/Hudson Yards', 'West Concourse', 'West Farms/Bronx River', 'West Village',
    'Westchester Village/Unionport', 'Westerleigh', 'Whitestone', 'Willets Point', 'Williamsbridge/Olinville',
    'Williamsburg (North Side)', 'Williamsburg (South Side)', 'Windsor Terrace', 'Woodhaven', 'Woodlawn/Wakefield',
    'Woodside', 'World Trade Center', 'Yorkville East', 'Yorkville West'
]

_CATEGORICAL_STR_VOCAB = {
    "pickup_zone_name": VOCABULARY,
    "dropoff_zone_name": VOCABULARY
}

_CATEGORICAL_NUM_BUCKETS = {
    "dayofweek": 7,
    "hourofday": 24,
    "weekofyear": 53
}

In [7]:
# Create an input function reading a file using the Dataset API
# Then provide the results to the Estimator API
def read_dataset(suffix, mode, batch_size):
    def _input_fn():
        client = BigQueryClient()
        read_session = client.read_session(
            parent="projects/" + PROJECT_ID,
            project_id=DATASET_GCP_PROJECT_ID,
            table_id="{}_{}".format(TABLE_ID, suffix), 
            dataset_id=DATASET_ID,
            selected_fields=INPUT_COLUMNS,
            output_types=[tf.string, tf.int64, tf.int64, tf.int64, tf.string, tf.string, tf.int64, tf.float64, tf.int64],
            requested_streams=10
        )
        
        def decode_row(records):
            features = records
            label = tf.cast(features.pop(LABEL_COLUMN), tf.float32)
            return features, label
        
        dataset = read_session.parallel_read_rows(sloppy=True).map(decode_row)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None  # indefinitely
            dataset = dataset.shuffle(buffer_size=1000*batch_size, seed=1234)
        else:
            num_epochs = 1  # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size)
        return dataset
    return _input_fn

## Feature engineering

In [25]:
def get_wide_deep():
    
    # One hot encode categorical features
    fc_dayofweek = tf.compat.v1.feature_column.categorical_column_with_identity(key="dayofweek",
                                                                                num_buckets=_CATEGORICAL_NUM_BUCKETS["dayofweek"])
    
    fc_hourofday = tf.compat.v1.feature_column.categorical_column_with_identity(key="hourofday",
                                                                                num_buckets=_CATEGORICAL_NUM_BUCKETS["hourofday"])
    
    fc_weekofyear = tf.compat.v1.feature_column.categorical_column_with_identity(key="weekofyear",
                                                                                 num_buckets=_CATEGORICAL_NUM_BUCKETS["weekofyear"])
    
    fc_pickuploc = tf.compat.v1.feature_column.categorical_column_with_vocabulary_list(key="pickup_zone_name",
                                                                                vocabulary_list=_CATEGORICAL_STR_VOCAB["pickup_zone_name"])
    
    fc_dropoffloc = tf.compat.v1.feature_column.categorical_column_with_vocabulary_list(key="dropoff_zone_name",
                                                                            vocabulary_list=_CATEGORICAL_STR_VOCAB["pickup_zone_name"])
    
    wide = [        
        # Sparse columns
        fc_dayofweek, fc_hourofday, fc_weekofyear,
        fc_pickuploc, fc_dropoffloc
    ]
    
    # mettre dans util
    def normalize_distance(col):
        mean = 3387.0
        std = 3891.02
        return (col - mean)/std
    
#     def normalize_passanger_cnt(col):
#         mean = 1.62
#         std = 1.26
#         return (col - mean)/std
    
    # Numerical column passanger_count
    fn_passenger_count = tf.compat.v1.feature_column.numeric_column(key="passenger_count")#, normalizer_fn=normalize_passanger_cnt)
    fn_distance = tf.compat.v1.feature_column.numeric_column(key="distance", normalizer_fn=normalize_distance)
    
    # Embedding_column to "group" together ...
    fc_embed_dayofweek = tf.compat.v1.feature_column.embedding_column(categorical_column=fc_dayofweek, dimension=NEMBEDS)#, max_norm=1)
    fc_embed_hourofday = tf.compat.v1.feature_column.embedding_column(categorical_column=fc_hourofday, dimension=NEMBEDS)#, max_norm=1)
    fc_embed_weekofyear = tf.compat.v1.feature_column.embedding_column(categorical_column=fc_weekofyear, dimension=NEMBEDS)#, max_norm=1)
    fc_embed_pickuploc = tf.compat.v1.feature_column.embedding_column(categorical_column=fc_pickuploc, dimension=NEMBEDS)#, max_norm=1)
    fc_embed_dropoffloc = tf.compat.v1.feature_column.embedding_column(categorical_column=fc_dropoffloc, dimension=NEMBEDS)#, max_norm=1)
    
    deep = [
        fn_passenger_count,
        fn_distance,
        fc_embed_dayofweek,
        fc_embed_hourofday,
        fc_embed_weekofyear,
        fc_embed_pickuploc,
        fc_embed_dropoffloc,
    ]
    
    return wide, deep

### Serving input receiver function

In [26]:
def serving_input_receiver_fn():
    receiver_tensors = {
        'dayofweek': tf.compat.v1.placeholder(dtype=tf.int64, shape=[None], name="dayofweek"),
        'hourofday': tf.compat.v1.placeholder(dtype=tf.int64, shape=[None], name="hourofday"),
        'weekofyear': tf.compat.v1.placeholder(dtype=tf.int64, shape=[None], name="weekofyear"),
        'pickup_zone_name': tf.compat.v1.placeholder(dtype=tf.string, shape=[None], name="pickup_zone_name"),
        'dropoff_zone_name': tf.compat.v1.placeholder(dtype=tf.string, shape=[None], name="dropoff_zone_name"),
        'passenger_count': tf.compat.v1.placeholder(dtype=tf.int64, shape=[None], name="passenger_count"),
        'distance': tf.compat.v1.placeholder(dtype=tf.float64, shape=[None], name="distance"),
        KEY_COLUMN: tf.compat.v1.placeholder_with_default(tf.constant(['no_key']), [None], name="uuid")
    }
    
    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in receiver_tensors.items()
    }
        
    return tf.estimator.export.ServingInputReceiver(features=features, receiver_tensors=receiver_tensors)

## Build and train model

In [20]:
def my_estimator(output_dir, throttle_secs, nnsize, batch_size, train_steps, eval_steps, eval_delay_secs):
    
    run_config = tf.estimator.RunConfig(save_checkpoints_secs=throttle_secs,
                                        tf_random_seed=2810,
                                        keep_checkpoint_max=3)
    
    # Add custom evaluation metric
    def my_rmse(labels, predictions):
        pred_values = tf.squeeze(input=predictions["predictions"], axis=-1)
        return {"rmse": tf.compat.v1.metrics.root_mean_squared_error(labels=labels, predictions=pred_values)}
    
    # Feature engineering
    wide, deep = get_wide_deep()
    
    estimator = tf.estimator.DNNLinearCombinedRegressor(
        model_dir=output_dir,
        linear_feature_columns=wide,
        dnn_feature_columns=deep,
        dnn_hidden_units=nnsize,
        batch_norm=True,
        dnn_dropout=0.1,
        config=run_config)
    
    estimator = tf.contrib.estimator.add_metrics(estimator=estimator, metric_fn=my_rmse)
    
    train_spec = tf.estimator.TrainSpec(
        input_fn=read_dataset('train', tf.estimator.ModeKeys.TRAIN, batch_size),
        max_steps=train_steps)
    
    exporter = tf.estimator.LatestExporter('exporter', serving_input_receiver_fn=serving_input_receiver_fn)
    
    eval_spec = tf.estimator.EvalSpec(
        input_fn=read_dataset('test', tf.estimator.ModeKeys.EVAL, 2**15),
        steps=eval_steps,
        start_delay_secs=eval_delay_secs,  # start evaluating after N seconds
        throttle_secs=throttle_secs,  # evaluate every N seconds
        exporters=exporter)
    
    return estimator, train_spec, eval_spec

### Train model

In [27]:
# Parameters
BUCKET = "edml"
OUTDIR = "gs://{}/ai-platform/models/edml_trainer_{}".format(BUCKET, datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))
THROTTLE_SECS = 300
NNSIZE = [128, 32, 4]
BATCH_SIZE = 128
TRAIN_STEPS = 100000 #5M per fare 3 apochs con batch_size 128 (input size 217M)
EVAL_STEPS = 3
EVAL_DECAY_SECS = 10

In [ ]:
# %%bash
# # gsutil rm gs://bucket/subdir/** will remove all objects under gs://bucket/subdir or any of its subdirectories.
# gsutil rm gs://edml/data/taxi-trips/model_test/** # start fresh each time

In [28]:
tf.compat.v1.summary.FileWriterCache.clear() # ensure filewriter cache is clear for TensorBoard events file
tf.compat.v1.logging.set_verbosity(v = tf.logging.INFO) # so loss is printed during training

In [29]:
estimator, train_spec, eval_spec = my_estimator(output_dir=OUTDIR, throttle_secs=THROTTLE_SECS, nnsize=NNSIZE, batch_size=BATCH_SIZE, train_steps=TRAIN_STEPS,
                                                eval_steps=EVAL_STEPS, eval_delay_secs=EVAL_DECAY_SECS)

INFO:tensorflow:Using config: {'_model_dir': 'gs://edml/ai-platform/models/edml_trainer_20200421_110923', '_tf_random_seed': 2810, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 300, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 3, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa1dc122150>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Using config: {'_model_dir': 'gs://ed

In [30]:
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 300.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into gs://edml/ai-platform/models/edml_trainer_20200421_110923/model.ckpt.
INFO:tensorflow:loss = 4233677.5, step = 0
INFO:tensorflow:global_step/sec: 28.2264
INFO:tensorflow:loss = 48058.49, step = 100 (3.544 sec)
INFO:tensorflow:global_step/sec: 35.7912
INFO:tensorflow:loss = 39977.812, step = 200 (2.7

({'average_loss': 4553.305,
  'label/mean': 18.21163,
  'loss': 149202700.0,
  'prediction/mean': 9.8668375,
  'rmse': 67.47818,
  'global_step': 100000},
 [b'gs://edml/ai-platform/models/edml_trainer_20200421_110923/export/exporter/1587470867'])

In [ ]:
# ({'average_loss': 4725.116,
#   'label/mean': 18.190115,
#   'loss': 154832600.0,
#   'prediction/mean': 0.98749757,
#   'rmse': 68.73948,
#   'global_step': 1000},
#  [b'gs://edml/ai-platform/models/edml_trainer_20200421_103536/export/exporter/1587465469'])